# Credit Card Fraud Detection - SPARK

In [ ]:
# -*- coding: utf-8 -*-
"""
Spark пример для бинарной классификации на Credit Card Fraud Detection
Запуск: spark-submit spark_example.py
Или: python spark_example.py (если Spark настроен в PYTHONPATH)
"""

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, 
                                  MulticlassClassificationEvaluator)
import time

def main():
    print("=" * 70)
    print("SPARK ML ПАЙПЛАЙН - Credit Card Fraud Detection") 
    print("=" * 70)
    
    # === ЭТАП 1: ИНИЦИАЛИЗАЦИЯ SPARK СЕССИИ ===
    print("\n1. ИНИЦИАЛИЗАЦИЯ SPARK СЕССИИ...")
    start_time = time.time()
    
    # Создаем SparkSession - точку входа в Spark
    spark = SparkSession.builder \
        .appName("CreditCardFraudDetection") \
        .config("spark.sql.adaptive.enabled", "true") \
        .getOrCreate()
    
    # Настройки для лучшей производительности
    spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")
    
    init_time = time.time() - start_time
    print(f"   ✓ Spark сессия создана за {init_time:.2f} секунд")
    print(f"   ✓ Версия Spark: {spark.version}")
    
    # === ЭТАП 2: ЗАГРУЗКА ДАННЫХ ===
    print("\n2. ЗАГРУЗКА ДАННЫХ В SPARK DATAFRAME...")
    start_time = time.time()
    
    try:
        # Читаем CSV в распределенный Spark DataFrame
        df = spark.read \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .csv("/work/dataset.csv")
        
        load_time = time.time() - start_time
        print(f"   ✓ Данные загружены за {load_time:.2f} секунд")
        
        # Показываем информацию о данных
        print(f"   ✓ Размер данных: {df.count():,} строк, {len(df.columns)} столбцов")
        print(f"   ✓ Partitions: {df.rdd.getNumPartitions()}")
        
    except Exception as e:
        print(f"   ✗ Ошибка загрузки: {e}")
        print("   Скачайте датасет с Kaggle: https://www.kaggle.com/mlg-ulb/creditcardfraud")
        spark.stop()
        return
    
    # === ЭТАП 3: АНАЛИЗ ДАННЫХ ===
    print("\n3. АНАЛИЗ ДАННЫХ В SPARK...")
    
    # Схема данных (типы столбцов)
    print("   Схема данных:")
    df.printSchema()
    
    # Распределение целевой переменной
    class_dist = df.groupBy("Class").agg(count("*").alias("count")).collect()
    print(f"   Распределение классов: { {row['Class']: row['count'] for row in class_dist} }")
    
    # === ЭТАП 4: ПОДГОТОВКА ДАННЫХ ===
    print("\n4. ПОДГОТОВКА ДАННЫХ ДЛЯ SPARK ML...")
    start_time = time.time()
    
    # В Spark ML все признаки должны быть собраны в один векторный столбец
    # Выбираем все столбцы кроме 'Class' как признаки
    feature_columns = [col for col in df.columns if col != 'Class']
    print(f"   Признаки ({len(feature_columns)}): {feature_columns}")
    
    # Создаем VectorAssembler - трансформатор для создания вектора признаков
    assembler = VectorAssembler(
        inputCols=feature_columns,   # Входные столбцы-признаки
        outputCol="features"         # Выходной столбец с вектором
    )
    
    # Применяем трансформатор к данным
    featured_df = assembler.transform(df)
    
    # Разделяем на обучающую и тестовую выборки
    train_df, test_df = featured_df.randomSplit(
        [0.7, 0.3],         # 70% train, 30% test
        seed=42              # Для воспроизводимости
    )
    
    prep_time = time.time() - start_time
    print(f"   ✓ Данные подготовлены за {prep_time:.2f} секунд")
    print(f"   ✓ Обучающая выборка: {train_df.count():,} samples")
    print(f"   ✓ Тестовая выборка: {test_df.count():,} samples")
    
    # Показываем как выглядят данные после преобразования
    print("\n   Пример данных с векторными признаками:")
    train_df.select("features", "Class").show(5, truncate=False)
    
    # === ЭТАП 5: ОБУЧЕНИЕ МОДЕЛИ ===
    print("\n5. ОБУЧЕНИЕ SPARK ML МОДЕЛИ (RandomForest)...")
    start_time = time.time()
    
    # Создаем модель RandomForest
    rf = RandomForestClassifier(
        featuresCol="features",      # Столбец с вектором признаков
        labelCol="Class",           # Столбец с целевой переменной
        numTrees=100,               # Количество деревьев
        maxDepth=10,                # Максимальная глубина
        seed=42,                    # Для воспроизводимости
        subsamplingRate=0.7         # Доля данных для каждого дерева
    )
    
    # Обучаем модель (распределенное обучение на кластере!)
    model = rf.fit(train_df)
    
    train_time = time.time() - start_time
    print(f"   ✓ Модель обучена за {train_time:.2f} секунд")
    print(f"   ✓ Обучено деревьев: {model.getNumTrees}")
    
    # === ЭТАП 6: ПРЕДСКАЗАНИЯ ===
    print("\n6. ПРЕДСКАЗАНИЯ НА ТЕСТОВЫХ ДАННЫХ...")
    start_time = time.time()
    
    # Делаем предсказания - модель добавляет новые столбцы к DataFrame
    predictions = model.transform(test_df)
    
    predict_time = time.time() - start_time
    print(f"   ✓ Предсказания сделаны за {predict_time:.2f} секунд")
    
    # Показываем структуру предсказаний
    print("\n   Структура предсказаний:")
    predictions.select("Class", "probability", "prediction").show(10, truncate=False)
    
    # === ЭТАП 7: ОЦЕНКА МОДЕЛИ ===
    print("\n7. ОЦЕНКА КАЧЕСТВА МОДЕЛИ:")
    print("-" * 50)
    
    # Evaluator для AUC-ROC (бинарная классификация)
    auc_evaluator = BinaryClassificationEvaluator(
        labelCol="Class",
        rawPredictionCol="rawPrediction",  # Столбец с "сырыми" предсказаниями
        metricName="areaUnderROC"
    )
    
    # Evaluator для других метрик
    multi_evaluator = MulticlassClassificationEvaluator(
        labelCol="Class",
        predictionCol="prediction"
    )
    
    # Вычисляем метрики
    auc = auc_evaluator.evaluate(predictions)
    accuracy = multi_evaluator.evaluate(predictions, {multi_evaluator.metricName: "accuracy"})
    precision = multi_evaluator.evaluate(predictions, {multi_evaluator.metricName: "weightedPrecision"})
    recall = multi_evaluator.evaluate(predictions, {multi_evaluator.metricName: "weightedRecall"})
    f1 = multi_evaluator.evaluate(predictions, {multi_evaluator.metricName: "f1"})
    
    # Выводим метрики
    print(f"   ТОЧНОСТЬ (Accuracy):  {accuracy:.4f}")
    print(f"   ТОЧНОСТЬ (Precision): {precision:.4f}")
    print(f"   ПОЛНОТА (Recall):     {recall:.4f}")
    print(f"   F1-MEASURE:          {f1:.4f}")
    print(f"   AUC-ROC:             {auc:.4f}")
    
    # Детальная статистика по классам
    print("\n   СТАТИСТИКА ПРЕДСКАЗАНИЙ:")
    predictions.groupBy("Class", "prediction").count().orderBy("Class", "prediction").show()
    
    # === ЭТАП 8: ЗАВЕРШЕНИЕ РАБОТЫ ===
    print("\n" + "=" * 70)
    print("ИТОГИ SPARK ПАЙПЛАЙНА:")
    print(f"   • Общее время выполнения: {init_time + load_time + prep_time + train_time + predict_time:.2f} сек")
    print(f"   • Время инициализации Spark: {init_time:.2f} сек")
    print(f"   • Размер данных: {df.count():,} строк × {len(df.columns)} столбцов")
    print(f"   • Partitions: {df.rdd.getNumPartitions()}")
    print(f"   • Лучшая метрика (AUC-ROC): {auc:.4f}")
    print("=" * 70)
    
    # Останавливаем Spark сессию
    spark.stop()
    print("   ✓ Spark сессия остановлена")

if __name__ == "__main__":
    main()

# Credit Card Fraud Detection - SPARK - Jupyter

In [1]:
# -*- coding: utf-8 -*-
"""
Spark пример для бинарной классификации на Credit Card Fraud Detection
Запуск: spark-submit spark_example.py
Или: python spark_example.py (если Spark настроен в PYTHONPATH)
"""

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, 
                                  MulticlassClassificationEvaluator)
import time

In [2]:
print("=" * 70)
print("SPARK ML ПАЙПЛАЙН - Credit Card Fraud Detection") 
print("=" * 70)

# === ЭТАП 1: ИНИЦИАЛИЗАЦИЯ SPARK СЕССИИ ===
print("\n1. ИНИЦИАЛИЗАЦИЯ SPARK СЕССИИ...")
start_time = time.time()


drivers = [
    "/home/jovyan/work/spark-jars/hadoop-aws-3.3.4.jar",             # S3
    "/home/jovyan/work/spark-jars/aws-java-sdk-bundle-1.12.262.jar", # S3
    "/home/jovyan/work/spark-jars/wildfly-openssl-1.0.7.Final.jar",  # S3
    "/home/jovyan/work/spark-jars/postgresql-42.6.0.jar",            # PostgreSQL
]

spark = (SparkSession.builder
         .appName("mustdayker-Spark")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .config("spark.sql.adaptive.enabled", "true")
         .getOrCreate()
        )

# Настройки для лучшей производительности
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")

init_time = time.time() - start_time
print(f"   ✓ Spark сессия создана за {init_time:.2f} секунд")
print(f"   ✓ Версия Spark: {spark.version}")

SPARK ML ПАЙПЛАЙН - Credit Card Fraud Detection

1. ИНИЦИАЛИЗАЦИЯ SPARK СЕССИИ...


25/11/03 16:52:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


   ✓ Spark сессия создана за 4.13 секунд
   ✓ Версия Spark: 3.5.0


In [3]:
# === ЭТАП 2: ЗАГРУЗКА ДАННЫХ ===
print("\n2. ЗАГРУЗКА ДАННЫХ В SPARK DATAFRAME...")
start_time = time.time()

try:
    # Читаем CSV в распределенный Spark DataFrame
    df = (spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv("/shared_data/creditcard.csv"))
    
    load_time = time.time() - start_time
    print(f"   ✓ Данные загружены за {load_time:.2f} секунд")
    
    # Показываем информацию о данных
    print(f"   ✓ Размер данных: {df.count():,} строк, {len(df.columns)} столбцов")
    print(f"   ✓ Partitions: {df.rdd.getNumPartitions()}")
    
except Exception as e:
    print(f"   ✗ Ошибка загрузки: {e}")
    print("   Скачайте датасет с Kaggle: https://www.kaggle.com/mlg-ulb/creditcardfraud")
    spark.stop()


2. ЗАГРУЗКА ДАННЫХ В SPARK DATAFRAME...


   ✓ Данные загружены за 6.65 секунд
   ✓ Размер данных: 284,807 строк, 31 столбцов
   ✓ Partitions: 10


In [6]:
# === ЭТАП 3: АНАЛИЗ ДАННЫХ ===
print("\n3. АНАЛИЗ ДАННЫХ В SPARK...")

# Схема данных (типы столбцов)
print("   Схема данных:")
df.printSchema()

# Распределение целевой переменной
class_dist = df.groupBy("Class").agg(count("*").alias("count")).collect()
print(f"   Распределение классов: { {row['Class']: row['count'] for row in class_dist} }")


3. АНАЛИЗ ДАННЫХ В SPARK...
   Схема данных:
root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: do

[Stage 6:=====>                                                    (1 + 9) / 10]

   Распределение классов: {1: 492, 0: 284315}


In [9]:
# === ЭТАП 4: ПОДГОТОВКА ДАННЫХ ===
print("\n4. ПОДГОТОВКА ДАННЫХ ДЛЯ SPARK ML...")
start_time = time.time()

# В Spark ML все признаки должны быть собраны в один векторный столбец
# Выбираем все столбцы кроме 'Class' как признаки
feature_columns = [col for col in df.columns if col != 'Class']
print(f"   Признаки ({len(feature_columns)}): {feature_columns}")

# Создаем VectorAssembler - трансформатор для создания вектора признаков
assembler = VectorAssembler(
    inputCols=feature_columns,   # Входные столбцы-признаки
    outputCol="features"         # Выходной столбец с вектором
)

# Применяем трансформатор к данным
featured_df = assembler.transform(df)

# Разделяем на обучающую и тестовую выборки
train_df, test_df = featured_df.randomSplit(
    [0.7, 0.3],         # 70% train, 30% test
    seed=42              # Для воспроизводимости
)

prep_time = time.time() - start_time
print(f"   ✓ Данные подготовлены за {prep_time:.2f} секунд")
print(f"   ✓ Обучающая выборка: {train_df.count():,} samples")
print(f"   ✓ Тестовая выборка: {test_df.count():,} samples")

# Показываем как выглядят данные после преобразования
print("\n   Пример данных с векторными признаками:")
train_df.select("features", "Class").show(5, truncate=False)


4. ПОДГОТОВКА ДАННЫХ ДЛЯ SPARK ML...
   Признаки (30): ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
   ✓ Данные подготовлены за 0.14 секунд


   ✓ Обучающая выборка: 199,727 samples
   ✓ Тестовая выборка: 85,080 samples

   Пример данных с векторными признаками:
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                                                                          

In [10]:
# === ЭТАП 5: ОБУЧЕНИЕ МОДЕЛИ ===
print("\n5. ОБУЧЕНИЕ SPARK ML МОДЕЛИ (RandomForest)...")
start_time = time.time()

# Создаем модель RandomForest
rf = RandomForestClassifier(
    featuresCol="features",      # Столбец с вектором признаков
    labelCol="Class",           # Столбец с целевой переменной
    numTrees=100,               # Количество деревьев
    maxDepth=10,                # Максимальная глубина
    seed=42,                    # Для воспроизводимости
    subsamplingRate=0.7         # Доля данных для каждого дерева
)

# Обучаем модель (распределенное обучение на кластере!)
model = rf.fit(train_df)

train_time = time.time() - start_time
print(f"   ✓ Модель обучена за {train_time:.2f} секунд")
print(f"   ✓ Обучено деревьев: {model.getNumTrees}")


5. ОБУЧЕНИЕ SPARK ML МОДЕЛИ (RandomForest)...


25/11/03 16:59:39 WARN DAGScheduler: Broadcasting large task binary with size 1326.9 KiB
25/11/03 16:59:40 WARN DAGScheduler: Broadcasting large task binary with size 1842.7 KiB
25/11/03 16:59:40 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
25/11/03 16:59:41 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB


   ✓ Модель обучена за 9.71 секунд
   ✓ Обучено деревьев: 100


In [11]:
# === ЭТАП 6: ПРЕДСКАЗАНИЯ ===
print("\n6. ПРЕДСКАЗАНИЯ НА ТЕСТОВЫХ ДАННЫХ...")
start_time = time.time()

# Делаем предсказания - модель добавляет новые столбцы к DataFrame
predictions = model.transform(test_df)

predict_time = time.time() - start_time
print(f"   ✓ Предсказания сделаны за {predict_time:.2f} секунд")

# Показываем структуру предсказаний
print("\n   Структура предсказаний:")
predictions.select("Class", "probability", "prediction").show(10, truncate=False)


6. ПРЕДСКАЗАНИЯ НА ТЕСТОВЫХ ДАННЫХ...
   ✓ Предсказания сделаны за 0.03 секунд

   Структура предсказаний:


25/11/03 17:00:03 WARN DAGScheduler: Broadcasting large task binary with size 1786.2 KiB


+-----+------------------------------------------+----------+
|Class|probability                               |prediction|
+-----+------------------------------------------+----------+
|0    |[0.9999447855271627,5.5214472837262884E-5]|0.0       |
|0    |[0.999958526320884,4.147367911593034E-5]  |0.0       |
|0    |[0.9999418196015177,5.818039848223632E-5] |0.0       |
|0    |[0.9999508182498783,4.9181750121580874E-5]|0.0       |
|0    |[0.9999452862175261,5.471378247389914E-5] |0.0       |
|0    |[0.9996615070532238,3.384929467762534E-4] |0.0       |
|0    |[0.9999151592755368,8.48407244632346E-5]  |0.0       |
|0    |[0.9999579160687273,4.2083931272711693E-5]|0.0       |
|0    |[0.9999306116679467,6.938833205323081E-5] |0.0       |
|0    |[0.999928245465879,7.175453412106176E-5]  |0.0       |
+-----+------------------------------------------+----------+
only showing top 10 rows



In [12]:
# === ЭТАП 7: ОЦЕНКА МОДЕЛИ ===
print("\n7. ОЦЕНКА КАЧЕСТВА МОДЕЛИ:")
print("-" * 50)

# Evaluator для AUC-ROC (бинарная классификация)
auc_evaluator = BinaryClassificationEvaluator(
    labelCol="Class",
    rawPredictionCol="rawPrediction",  # Столбец с "сырыми" предсказаниями
    metricName="areaUnderROC"
)

# Evaluator для других метрик
multi_evaluator = MulticlassClassificationEvaluator(
    labelCol="Class",
    predictionCol="prediction"
)

# Вычисляем метрики
auc = auc_evaluator.evaluate(predictions)
accuracy = multi_evaluator.evaluate(predictions, {multi_evaluator.metricName: "accuracy"})
precision = multi_evaluator.evaluate(predictions, {multi_evaluator.metricName: "weightedPrecision"})
recall = multi_evaluator.evaluate(predictions, {multi_evaluator.metricName: "weightedRecall"})
f1 = multi_evaluator.evaluate(predictions, {multi_evaluator.metricName: "f1"})

# Выводим метрики
print(f"   ТОЧНОСТЬ (Accuracy):  {accuracy:.4f}")
print(f"   ТОЧНОСТЬ (Precision): {precision:.4f}")
print(f"   ПОЛНОТА (Recall):     {recall:.4f}")
print(f"   F1-MEASURE:          {f1:.4f}")
print(f"   AUC-ROC:             {auc:.4f}")

# Детальная статистика по классам
print("\n   СТАТИСТИКА ПРЕДСКАЗАНИЙ:")
predictions.groupBy("Class", "prediction").count().orderBy("Class", "prediction").show()

# === ЭТАП 8: ЗАВЕРШЕНИЕ РАБОТЫ ===
print("\n" + "=" * 70)
print("ИТОГИ SPARK ПАЙПЛАЙНА:")
print(f"   • Общее время выполнения: {init_time + load_time + prep_time + train_time + predict_time:.2f} сек")
print(f"   • Время инициализации Spark: {init_time:.2f} сек")
print(f"   • Размер данных: {df.count():,} строк × {len(df.columns)} столбцов")
print(f"   • Partitions: {df.rdd.getNumPartitions()}")
print(f"   • Лучшая метрика (AUC-ROC): {auc:.4f}")
print("=" * 70)

# Останавливаем Spark сессию
spark.stop()
print("   ✓ Spark сессия остановлена")


7. ОЦЕНКА КАЧЕСТВА МОДЕЛИ:
--------------------------------------------------


25/11/03 17:00:29 WARN DAGScheduler: Broadcasting large task binary with size 1785.5 KiB
25/11/03 17:00:31 WARN DAGScheduler: Broadcasting large task binary with size 1797.9 KiB
25/11/03 17:00:32 WARN DAGScheduler: Broadcasting large task binary with size 1797.9 KiB
25/11/03 17:00:32 WARN DAGScheduler: Broadcasting large task binary with size 1797.9 KiB
25/11/03 17:00:33 WARN DAGScheduler: Broadcasting large task binary with size 1797.9 KiB
25/11/03 17:00:33 WARN DAGScheduler: Broadcasting large task binary with size 1793.1 KiB


   ТОЧНОСТЬ (Accuracy):  0.9995
   ТОЧНОСТЬ (Precision): 0.9995
   ПОЛНОТА (Recall):     0.9995
   F1-MEASURE:          0.9995
   AUC-ROC:             0.9762

   СТАТИСТИКА ПРЕДСКАЗАНИЙ:


25/11/03 17:00:34 WARN DAGScheduler: Broadcasting large task binary with size 1768.4 KiB
                                                                                

+-----+----------+-----+
|Class|prediction|count|
+-----+----------+-----+
|    0|       0.0|84915|
|    0|       1.0|    4|
|    1|       0.0|   39|
|    1|       1.0|  122|
+-----+----------+-----+


ИТОГИ SPARK ПАЙПЛАЙНА:
   • Общее время выполнения: 20.67 сек
   • Время инициализации Spark: 4.13 сек
   • Размер данных: 284,807 строк × 31 столбцов
   • Partitions: 10
   • Лучшая метрика (AUC-ROC): 0.9762
   ✓ Spark сессия остановлена


# Housing - SKLEARN

In [ ]:
# -*- coding: utf-8 -*-
"""
Spark пример для РЕГРЕССИИ на California Housing Dataset
Запуск: spark-submit spark_regression.py
"""

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, stddev, count, when, isnan
from pyspark.ml.feature import VectorAssembler, Imputer
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import time

def main():
    print("=" * 70)
    print("SPARK РЕГРЕССИЯ - California Housing Dataset")
    print("=" * 70)
    
    # === ЭТАП 1: ИНИЦИАЛИЗАЦИЯ SPARK СЕССИИ ===
    print("\n1. ИНИЦИАЛИЗАЦИЯ SPARK СЕССИИ...")
    start_time = time.time()
    
    spark = SparkSession.builder \
        .appName("CaliforniaHousingRegression") \
        .config("spark.sql.adaptive.enabled", "true") \
        .getOrCreate()
    
    init_time = time.time() - start_time
    print(f"   ✓ Spark сессия создана за {init_time:.2f} секунд")
    
    # === ЭТАП 2: ЗАГРУЗКА И ОЧИСТКА ДАННЫХ ===
    print("\n2. ЗАГРУЗКА И ОЧИСТКА ДАННЫХ...")
    start_time = time.time()
    
    try:
        df = spark.read \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .csv("/work/california_housing.csv")
        
        # Удаляем текстовый признак 'ocean_proximity'
        if 'ocean_proximity' in df.columns:
            df = df.drop('ocean_proximity')
            print("   ✓ Текстовый признак 'ocean_proximity' удален")
        
        load_time = time.time() - start_time
        print(f"   ✓ Данные загружены за {load_time:.2f} секунд")
        print(f"   ✓ Размер данных: {df.count():,} строк, {len(df.columns)} столбцов")
        print(f"   ✓ Столбцы после очистки: {df.columns}")
        
    except Exception as e:
        print(f"   ✗ Ошибка загрузки: {e}")
        print("   Скачайте датасет: https://www.kaggle.com/datasets/camnugent/california-housing-prices")
        spark.stop()
        return
    
    # === ЭТАП 3: АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ ===
    print("\n3. АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ...")
    
    # Проверяем пропущенные значения для каждого столбца
    print("   Пропущенные значения до обработки:")
    missing_stats = []
    for column in df.columns:
        missing_count = df.filter(col(column).isNull() | isnan(col(column))).count()
        total_count = df.count()
        if missing_count > 0:
            missing_percent = (missing_count / total_count) * 100
            print(f"     {column}: {missing_count} пропусков ({missing_percent:.1f}%)")
            missing_stats.append((column, missing_count, missing_percent))
    
    if not missing_stats:
        print("     ✓ Пропущенных значений не обнаружено")
    
    # Статистика по целевой переменной
    target_col = "median_house_value"
    stats = df.select(
        mean(target_col).alias("mean"),
        stddev(target_col).alias("stddev"),
        col(target_col).min().alias("min"),
        col(target_col).max().alias("max")
    ).collect()[0]
    
    print(f"   Целевая переменная '{target_col}':")
    print(f"     Min: ${stats['min']:,.0f}, Max: ${stats['max']:,.0f}")
    print(f"     Mean: ${stats['mean']:,.0f}, Std: ${stats['stddev']:,.0f}")
    
    # === ЭТАП 4: ОБРАБОТКА ПРОПУЩЕННЫХ ЗНАЧЕНИЙ ===
    print("\n4. ОБРАБОТКА ПРОПУЩЕННЫХ ЗНАЧЕНИЙ...")
    start_time = time.time()
    
    # В Spark ML используем Imputer для заполнения пропусков медианой
    # Выбираем только числовые столбцы для импьютации
    numeric_columns = [col_name for col_name, data_type in df.dtypes 
                      if data_type in ['int', 'double', 'float']]
    
    print(f"   Числовые столбцы для обработки: {numeric_columns}")
    
    # Создаем Imputer для заполнения пропусков медианой
    imputer = Imputer(
        inputCols=numeric_columns,
        outputCols=[f"{col}_imputed" for col in numeric_columns],
        strategy="median"
    )
    
    # Обучаем импьютер и преобразуем данные
    imputer_model = imputer.fit(df)
    df_imputed = imputer_model.transform(df)
    
    # Удаляем оригинальные столбцы и переименовываем импутированные
    for col_name in numeric_columns:
        df_imputed = df_imputed.drop(col_name).withColumnRenamed(f"{col_name}_imputed", col_name)
    
    # Проверяем результат
    missing_after = 0
    for column in df_imputed.columns:
        missing_count = df_imputed.filter(col(column).isNull() | isnan(col(column))).count()
        missing_after += missing_count
    
    print(f"   ✓ Пропуски заполнены медианой")
    print(f"   ✓ Пропущенных значений после обработки: {missing_after}")
    
    # Показываем какие столбцы были обработаны
    for col_name, missing_count, missing_percent in missing_stats:
        print(f"     {col_name}: {missing_count} пропусков → заполнено медианой")
    
    prep_time = time.time() - start_time
    print(f"   ✓ Обработка пропусков выполнена за {prep_time:.2f} секунд")
    
    # === ЭТАП 5: ПОДГОТОВКА ДАННЫХ ДЛЯ SPARK ML ===
    print("\n5. ПОДГОТОВКА ДАННЫХ ДЛЯ SPARK ML...")
    start_time = time.time()
    
    # В Spark ML для регрессии нужен векторный столбец с признаками
    feature_columns = [col for col in df_imputed.columns if col != target_col]
    print(f"   Числовые признаки ({len(feature_columns)}): {feature_columns}")
    
    # Создаем VectorAssembler для объединения признаков в вектор
    assembler = VectorAssembler(
        inputCols=feature_columns,
        outputCol="features"
    )
    
    # Применяем трансформатор
    featured_df = assembler.transform(df_imputed)
    
    # Разделяем на обучающую и тестовую выборки
    train_df, test_df = featured_df.randomSplit(
        [0.7, 0.3],    # 70% train, 30% test
        seed=42
    )
    
    assemble_time = time.time() - start_time
    print(f"   ✓ Данные подготовлены за {assemble_time:.2f} секунд")
    print(f"   ✓ Обучающая выборка: {train_df.count():,} samples")
    print(f"   ✓ Тестовая выборка: {test_df.count():,} samples")
    
    # Показываем пример данных с векторными признаками
    print("\n   Пример данных с векторными признаками:")
    train_df.select("features", target_col).show(5, truncate=False)
    
    # === ЭТАП 6: ОБУЧЕНИЕ МОДЕЛИ ГРАДИЕНТНЫЙ БУСТИНГ ===
    print("\n6. ОБУЧЕНИЕ SPARK ML МОДЕЛИ (GBTRegressor)...")
    start_time = time.time()
    
    # Создаем модель Gradient Boosted Trees для регрессии
    gbt = GBTRegressor(
        featuresCol="features",      # Столбец с вектором признаков
        labelCol=target_col,         # Столбец с целевой переменной
        maxIter=100,                 # Количество деревьев (итераций)
        maxDepth=3,                  # Максимальная глубина деревьев
        stepSize=0.1,               # Темп обучения (learning rate)
        seed=42,                     # Для воспроизводимости
        subsamplingRate=0.8         # Доля данных для каждого дерева
    )
    
    # Обучаем модель
    model = gbt.fit(train_df)
    
    train_time = time.time() - start_time
    print(f"   ✓ Модель обучена за {train_time:.2f} секунд")
    print(f"   ✓ Обучено деревьев: {model.getNumTrees}")
    
    # === ЭТАП 7: ПРЕДСКАЗАНИЯ ===
    print("\n7. ПРЕДСКАЗАНИЯ НА ТЕСТОВЫХ ДАННЫХ...")
    start_time = time.time()
    
    # Делаем предсказания
    predictions = model.transform(test_df)
    
    predict_time = time.time() - start_time
    print(f"   ✓ Предсказания сделаны за {predict_time:.2f} секунд")
    
    # Показываем структуру предсказаний
    print("\n   Пример предсказаний:")
    predictions.select(target_col, "prediction").show(10, truncate=False)
    
    # === ЭТАП 8: ОЦЕНКА МОДЕЛИ РЕГРЕССИИ ===
    print("\n8. ОЦЕНКА КАЧЕСТВА РЕГРЕССИИ:")
    print("-" * 50)
    
    # Создаем evaluators для разных метрик регрессии
    evaluator_mse = RegressionEvaluator(
        labelCol=target_col,
        predictionCol="prediction",
        metricName="mse"
    )
    
    evaluator_rmse = RegressionEvaluator(
        labelCol=target_col,
        predictionCol="prediction", 
        metricName="rmse"
    )
    
    evaluator_r2 = RegressionEvaluator(
        labelCol=target_col,
        predictionCol="prediction",
        metricName="r2"
    )
    
    evaluator_mae = RegressionEvaluator(
        labelCol=target_col,
        predictionCol="prediction",
        metricName="mae"
    )
    
    # Вычисляем все метрики
    mse = evaluator_mse.evaluate(predictions)
    rmse = evaluator_rmse.evaluate(predictions)
    r2 = evaluator_r2.evaluate(predictions)
    mae = evaluator_mae.evaluate(predictions)
    
    # Выводим метрики
    print(f"   СРЕДНЯЯ АБСОЛЮТНАЯ ОШИБКА (MAE):    ${mae:,.0f}")
    print(f"   СРЕДНЯЯ КВАДРАТИЧНАЯ ОШИБКА (MSE):  ${mse:,.0f}")
    print(f"   КОРЕНЬ ИЗ MSE (RMSE):               ${rmse:,.0f}")
    print(f"   КОЭФФИЦИЕНТ ДЕТЕРМИНАЦИИ (R²):      {r2:.4f}")
    
    # Интерпретация R²
    print(f"\n   R² = {r2:.1%} - модель объясняет {r2:.1%} дисперсии целевой переменной")
    
    # === ЭТАП 9: АНАЛИЗ ВАЖНОСТИ ПРИЗНАКОВ ===
    print("\n9. ВАЖНОСТЬ ПРИЗНАКОВ:")
    feature_importances = list(zip(feature_columns, model.featureImportances))
    sorted_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)
    
    print("   Признак -> Важность:")
    for feature, importance in sorted_importances:
        print(f"   {feature:20} {importance:.4f}")
    
    # === ЭТАП 10: ЗАВЕРШЕНИЕ РАБОТЫ ===
    print("\n" + "=" * 70)
    print("ИТОГИ SPARK РЕГРЕССИИ:")
    print(f"   • Общее время выполнения: {init_time + load_time + prep_time + assemble_time + train_time + predict_time:.2f} сек")
    print(f"   • Время инициализации Spark: {init_time:.2f} сек")
    print(f"   • Размер данных: {df.count():,} строк × {len(df.columns)} столбцов")
    print(f"   • Обработано пропусков: {sum(missing[1] for missing in missing_stats)} значений")
    print(f"   • Все признаки числовые: {df_imputed.columns}")
    print(f"   • Лучшая метрика (R²): {r2:.4f}")
    print(f"   • Ошибка предсказания: ±${rmse:,.0f}")
    print("=" * 70)
    
    # Останавливаем Spark сессию
    spark.stop()
    print("   ✓ Spark сессия остановлена")

if __name__ == "__main__":
    main()

# Housing - SKLEARN - Jupyter

In [7]:
# -*- coding: utf-8 -*-
"""
Spark пример для РЕГРЕССИИ на California Housing Dataset
Запуск: spark-submit spark_regression.py
"""

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, stddev, count, when, isnan
from pyspark.ml.feature import VectorAssembler, Imputer
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import time

from pyspark.sql.functions import mean, stddev, min as spark_min, max as spark_max, col

In [2]:
print("=" * 70)
print("SPARK РЕГРЕССИЯ - California Housing Dataset")
print("=" * 70)

# === ЭТАП 1: ИНИЦИАЛИЗАЦИЯ SPARK СЕССИИ ===
print("\n1. ИНИЦИАЛИЗАЦИЯ SPARK СЕССИИ...")
start_time = time.time()

drivers = [
    "/home/jovyan/work/spark-jars/hadoop-aws-3.3.4.jar",             # S3
    "/home/jovyan/work/spark-jars/aws-java-sdk-bundle-1.12.262.jar", # S3
    "/home/jovyan/work/spark-jars/wildfly-openssl-1.0.7.Final.jar",  # S3
    "/home/jovyan/work/spark-jars/postgresql-42.6.0.jar",            # PostgreSQL
]

spark = (SparkSession.builder
         .appName("mustdayker-Spark")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .config("spark.sql.adaptive.enabled", "true")
         .getOrCreate()
        )

init_time = time.time() - start_time
print(f"   ✓ Spark сессия создана за {init_time:.2f} секунд")

SPARK РЕГРЕССИЯ - California Housing Dataset

1. ИНИЦИАЛИЗАЦИЯ SPARK СЕССИИ...


25/11/03 20:31:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


   ✓ Spark сессия создана за 3.43 секунд


In [3]:
# === ЭТАП 2: ЗАГРУЗКА И ОЧИСТКА ДАННЫХ ===
print("\n2. ЗАГРУЗКА И ОЧИСТКА ДАННЫХ...")
start_time = time.time()

try:
    df = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv("/shared_data/housing.csv")
    
    # Удаляем текстовый признак 'ocean_proximity'
    if 'ocean_proximity' in df.columns:
        df = df.drop('ocean_proximity')
        print("   ✓ Текстовый признак 'ocean_proximity' удален")
    
    load_time = time.time() - start_time
    print(f"   ✓ Данные загружены за {load_time:.2f} секунд")
    print(f"   ✓ Размер данных: {df.count():,} строк, {len(df.columns)} столбцов")
    print(f"   ✓ Столбцы после очистки: {df.columns}")
    
except Exception as e:
    print(f"   ✗ Ошибка загрузки: {e}")
    print("   Скачайте датасет: https://www.kaggle.com/datasets/camnugent/california-housing-prices")
    spark.stop()


2. ЗАГРУЗКА И ОЧИСТКА ДАННЫХ...


   ✓ Текстовый признак 'ocean_proximity' удален
   ✓ Данные загружены за 6.72 секунд
   ✓ Размер данных: 20,640 строк, 9 столбцов
   ✓ Столбцы после очистки: ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']


In [4]:
# === ЭТАП 3: АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ ===
print("\n3. АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ...")

# Проверяем пропущенные значения для каждого столбца
print("   Пропущенные значения до обработки:")
missing_stats = []
for column in df.columns:
    missing_count = df.filter(col(column).isNull() | isnan(col(column))).count()
    total_count = df.count()
    if missing_count > 0:
        missing_percent = (missing_count / total_count) * 100
        print(f"     {column}: {missing_count} пропусков ({missing_percent:.1f}%)")
        missing_stats.append((column, missing_count, missing_percent))

if not missing_stats:
    print("     ✓ Пропущенных значений не обнаружено")


3. АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ...
   Пропущенные значения до обработки:
     total_bedrooms: 207 пропусков (1.0%)


In [8]:
# Статистика по целевой переменной
target_col = "median_house_value"
# stats = df.select(
#     mean(target_col).alias("mean"),
#     stddev(target_col).alias("stddev"),
#     col(target_col).min().alias("min"),
#     col(target_col).max().alias("max")
# ).collect()[0]

stats = df.select(
    mean(col(target_col)).alias("mean"),
    stddev(col(target_col)).alias("stddev"),
    spark_min(col(target_col)).alias("min"),
    spark_max(col(target_col)).alias("max")
).collect()[0]


print(f"   Целевая переменная '{target_col}':")
print(f"     Min: ${stats['min']:,.0f}, Max: ${stats['max']:,.0f}")
print(f"     Mean: ${stats['mean']:,.0f}, Std: ${stats['stddev']:,.0f}")

   Целевая переменная 'median_house_value':
     Min: $14,999, Max: $500,001
     Mean: $206,856, Std: $115,396


In [9]:
# === ЭТАП 4: ОБРАБОТКА ПРОПУЩЕННЫХ ЗНАЧЕНИЙ ===
print("\n4. ОБРАБОТКА ПРОПУЩЕННЫХ ЗНАЧЕНИЙ...")
start_time = time.time()

# В Spark ML используем Imputer для заполнения пропусков медианой
# Выбираем только числовые столбцы для импьютации
numeric_columns = [col_name for col_name, data_type in df.dtypes 
                  if data_type in ['int', 'double', 'float']]

print(f"   Числовые столбцы для обработки: {numeric_columns}")

# Создаем Imputer для заполнения пропусков медианой
imputer = Imputer(
    inputCols=numeric_columns,
    outputCols=[f"{col}_imputed" for col in numeric_columns],
    strategy="median"
)

# Обучаем импьютер и преобразуем данные
imputer_model = imputer.fit(df)
df_imputed = imputer_model.transform(df)

# Удаляем оригинальные столбцы и переименовываем импутированные
for col_name in numeric_columns:
    df_imputed = df_imputed.drop(col_name).withColumnRenamed(f"{col_name}_imputed", col_name)

# Проверяем результат
missing_after = 0
for column in df_imputed.columns:
    missing_count = df_imputed.filter(col(column).isNull() | isnan(col(column))).count()
    missing_after += missing_count

print(f"   ✓ Пропуски заполнены медианой")
print(f"   ✓ Пропущенных значений после обработки: {missing_after}")

# Показываем какие столбцы были обработаны
for col_name, missing_count, missing_percent in missing_stats:
    print(f"     {col_name}: {missing_count} пропусков → заполнено медианой")

prep_time = time.time() - start_time
print(f"   ✓ Обработка пропусков выполнена за {prep_time:.2f} секунд")


4. ОБРАБОТКА ПРОПУЩЕННЫХ ЗНАЧЕНИЙ...
   Числовые столбцы для обработки: ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']


   ✓ Пропуски заполнены медианой
   ✓ Пропущенных значений после обработки: 0
     total_bedrooms: 207 пропусков → заполнено медианой
   ✓ Обработка пропусков выполнена за 2.50 секунд


In [10]:
# === ЭТАП 5: ПОДГОТОВКА ДАННЫХ ДЛЯ SPARK ML ===
print("\n5. ПОДГОТОВКА ДАННЫХ ДЛЯ SPARK ML...")
start_time = time.time()

# В Spark ML для регрессии нужен векторный столбец с признаками
feature_columns = [col for col in df_imputed.columns if col != target_col]
print(f"   Числовые признаки ({len(feature_columns)}): {feature_columns}")

# Создаем VectorAssembler для объединения признаков в вектор
assembler = VectorAssembler(
    inputCols=feature_columns,
    outputCol="features"
)

# Применяем трансформатор
featured_df = assembler.transform(df_imputed)

# Разделяем на обучающую и тестовую выборки
train_df, test_df = featured_df.randomSplit(
    [0.7, 0.3],    # 70% train, 30% test
    seed=42
)

assemble_time = time.time() - start_time
print(f"   ✓ Данные подготовлены за {assemble_time:.2f} секунд")
print(f"   ✓ Обучающая выборка: {train_df.count():,} samples")
print(f"   ✓ Тестовая выборка: {test_df.count():,} samples")

# Показываем пример данных с векторными признаками
print("\n   Пример данных с векторными признаками:")
train_df.select("features", target_col).show(5, truncate=False)


5. ПОДГОТОВКА ДАННЫХ ДЛЯ SPARK ML...
   Числовые признаки (8): ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']
   ✓ Данные подготовлены за 0.10 секунд
   ✓ Обучающая выборка: 14,509 samples
   ✓ Тестовая выборка: 6,131 samples

   Пример данных с векторными признаками:
+-----------------------------------------------------+------------------+
|features                                             |median_house_value|
+-----------------------------------------------------+------------------+
|[-124.35,40.54,52.0,1820.0,300.0,806.0,270.0,3.0147] |94600.0           |
|[-124.3,41.8,19.0,2672.0,552.0,1298.0,478.0,1.9797]  |85800.0           |
|[-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179]|79000.0           |
|[-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571] |111400.0          |
|[-124.25,40.28,32.0,1430.0,419.0,434.0,187.0,1.9417] |76100.0           |
+--------------------------------------------------

In [11]:
# === ЭТАП 6: ОБУЧЕНИЕ МОДЕЛИ ГРАДИЕНТНЫЙ БУСТИНГ ===
print("\n6. ОБУЧЕНИЕ SPARK ML МОДЕЛИ (GBTRegressor)...")
start_time = time.time()

# Создаем модель Gradient Boosted Trees для регрессии
gbt = GBTRegressor(
    featuresCol="features",      # Столбец с вектором признаков
    labelCol=target_col,         # Столбец с целевой переменной
    maxIter=100,                 # Количество деревьев (итераций)
    maxDepth=3,                  # Максимальная глубина деревьев
    stepSize=0.1,               # Темп обучения (learning rate)
    seed=42,                     # Для воспроизводимости
    subsamplingRate=0.8         # Доля данных для каждого дерева
)

# Обучаем модель
model = gbt.fit(train_df)

train_time = time.time() - start_time
print(f"   ✓ Модель обучена за {train_time:.2f} секунд")
print(f"   ✓ Обучено деревьев: {model.getNumTrees}")


6. ОБУЧЕНИЕ SPARK ML МОДЕЛИ (GBTRegressor)...
   ✓ Модель обучена за 15.09 секунд
   ✓ Обучено деревьев: 100


In [12]:
# === ЭТАП 7: ПРЕДСКАЗАНИЯ ===
print("\n7. ПРЕДСКАЗАНИЯ НА ТЕСТОВЫХ ДАННЫХ...")
start_time = time.time()

# Делаем предсказания
predictions = model.transform(test_df)

predict_time = time.time() - start_time
print(f"   ✓ Предсказания сделаны за {predict_time:.2f} секунд")

# Показываем структуру предсказаний
print("\n   Пример предсказаний:")
predictions.select(target_col, "prediction").show(10, truncate=False)


7. ПРЕДСКАЗАНИЯ НА ТЕСТОВЫХ ДАННЫХ...
   ✓ Предсказания сделаны за 0.10 секунд

   Пример предсказаний:


+------------------+------------------+
|median_house_value|prediction        |
+------------------+------------------+
|103600.0          |98551.11233609286 |
|106700.0          |139656.5624114286 |
|73200.0           |90367.54730552975 |
|78300.0           |100622.5160580698 |
|90100.0           |133051.29064697694|
|69000.0           |95306.63816421998 |
|70000.0           |90113.78742989336 |
|67000.0           |71014.71124352298 |
|70500.0           |61170.070879167695|
|86400.0           |101184.34011455691|
+------------------+------------------+
only showing top 10 rows



In [13]:
# === ЭТАП 8: ОЦЕНКА МОДЕЛИ РЕГРЕССИИ ===
print("\n8. ОЦЕНКА КАЧЕСТВА РЕГРЕССИИ:")
print("-" * 50)

# Создаем evaluators для разных метрик регрессии
evaluator_mse = RegressionEvaluator(
    labelCol=target_col,
    predictionCol="prediction",
    metricName="mse"
)

evaluator_rmse = RegressionEvaluator(
    labelCol=target_col,
    predictionCol="prediction", 
    metricName="rmse"
)

evaluator_r2 = RegressionEvaluator(
    labelCol=target_col,
    predictionCol="prediction",
    metricName="r2"
)

evaluator_mae = RegressionEvaluator(
    labelCol=target_col,
    predictionCol="prediction",
    metricName="mae"
)

# Вычисляем все метрики
mse = evaluator_mse.evaluate(predictions)
rmse = evaluator_rmse.evaluate(predictions)
r2 = evaluator_r2.evaluate(predictions)
mae = evaluator_mae.evaluate(predictions)

# Выводим метрики
print(f"   СРЕДНЯЯ АБСОЛЮТНАЯ ОШИБКА (MAE):    ${mae:,.0f}")
print(f"   СРЕДНЯЯ КВАДРАТИЧНАЯ ОШИБКА (MSE):  ${mse:,.0f}")
print(f"   КОРЕНЬ ИЗ MSE (RMSE):               ${rmse:,.0f}")
print(f"   КОЭФФИЦИЕНТ ДЕТЕРМИНАЦИИ (R²):      {r2:.4f}")

# Интерпретация R²
print(f"\n   R² = {r2:.1%} - модель объясняет {r2:.1%} дисперсии целевой переменной")


8. ОЦЕНКА КАЧЕСТВА РЕГРЕССИИ:
--------------------------------------------------
   СРЕДНЯЯ АБСОЛЮТНАЯ ОШИБКА (MAE):    $36,862
   СРЕДНЯЯ КВАДРАТИЧНАЯ ОШИБКА (MSE):  $2,808,035,892
   КОРЕНЬ ИЗ MSE (RMSE):               $52,991
   КОЭФФИЦИЕНТ ДЕТЕРМИНАЦИИ (R²):      0.7921

   R² = 79.2% - модель объясняет 79.2% дисперсии целевой переменной


In [14]:
# === ЭТАП 9: АНАЛИЗ ВАЖНОСТИ ПРИЗНАКОВ ===
print("\n9. ВАЖНОСТЬ ПРИЗНАКОВ:")
feature_importances = list(zip(feature_columns, model.featureImportances))
sorted_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)

print("   Признак -> Важность:")
for feature, importance in sorted_importances:
    print(f"   {feature:20} {importance:.4f}")


9. ВАЖНОСТЬ ПРИЗНАКОВ:
   Признак -> Важность:
   longitude            0.2918
   median_income        0.2479
   latitude             0.2334
   population           0.0719
   total_bedrooms       0.0569
   housing_median_age   0.0543
   households           0.0276
   total_rooms          0.0161


In [15]:
# === ЭТАП 10: ЗАВЕРШЕНИЕ РАБОТЫ ===
print("\n" + "=" * 70)
print("ИТОГИ SPARK РЕГРЕССИИ:")
print(f"   • Общее время выполнения: {init_time + load_time + prep_time + assemble_time + train_time + predict_time:.2f} сек")
print(f"   • Время инициализации Spark: {init_time:.2f} сек")
print(f"   • Размер данных: {df.count():,} строк × {len(df.columns)} столбцов")
print(f"   • Обработано пропусков: {sum(missing[1] for missing in missing_stats)} значений")
print(f"   • Все признаки числовые: {df_imputed.columns}")
print(f"   • Лучшая метрика (R²): {r2:.4f}")
print(f"   • Ошибка предсказания: ±${rmse:,.0f}")
print("=" * 70)


ИТОГИ SPARK РЕГРЕССИИ:
   • Общее время выполнения: 27.94 сек
   • Время инициализации Spark: 3.43 сек
   • Размер данных: 20,640 строк × 9 столбцов
   • Обработано пропусков: 207 значений
   • Все признаки числовые: ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']
   • Лучшая метрика (R²): 0.7921
   • Ошибка предсказания: ±$52,991


In [16]:
# Останавливаем Spark сессию
spark.stop()
print("   ✓ Spark сессия остановлена")

   ✓ Spark сессия остановлена
